In [1]:
import os
import glob
#import tqdm
import numpy as np
#import pandas as pd
import multiprocessing
import pydicom as dicom
import nibabel as nib
#from keras import utils as kutils
from matplotlib import pyplot as plt

def load_image(label_fpath, transpose=False):
    #encode_name = label_fpath[-39: -7]
    label_data = nib.load(label_fpath)
    label_array = label_data.get_fdata()
    if transpose:
        label_array = np.transpose(label_array, axes=(2, 1, 0))
    return  label_array#, encode_name

In [2]:
cd ..

/home/u/wartytw


In [3]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')

In [4]:
image_path = "/data2/pancreas/Nifti_data/image"
label_path = "/data2/pancreas/Nifti_data/label"

resec_train = np.load("resec_train.npy")
resec_val = np.load("resec_val.npy")
y_train = np.load("y_train.npy")
y_val = np.load("y_val.npy")

resec = np.concatenate([resec_train,resec_val], axis = 0)
y = np.concatenate([y_train, y_val], axis = 0)

box_tumor = np.load("box_tumor.npy")
box_pancreas = np.load("box_pancreas.npy")

import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')

In [5]:
from keras.utils import to_categorical
from scipy import ndimage


#from label(whole pancreas) to tumor_only masking
def extract_tumor(tmp_slide):
    t_tmp_slide = np.zeros((tmp_slide.shape[0], tmp_slide.shape[1]))
    for i in range(tmp_slide.shape[0]):
        for j in range(tmp_slide.shape[1]):
            if(tmp_slide[i, j] == 2):
                t_tmp_slide[i, j] = 2
    return t_tmp_slide


Using TensorFlow backend.


In [6]:
#masking for pancreas-tumor
def masking_dilation_erosion(tmp_slide, t_tmp_slide):
    tt_tmp_slide = np.zeros((tmp_slide.shape[0], tmp_slide.shape[1]))
    for i in range(tmp_slide.shape[0]):
        for j in range(tmp_slide.shape[1]):
            if(tmp_slide[i, j] != 0 and t_tmp_slide[i, j] != 1):
                tt_tmp_slide[i, j] = 1
    return tt_tmp_slide


In [7]:
def masking_image(label, img):
    ll = np.zeros((label.shape[0], label.shape[1]))
    for i in range(label.shape[0]):
        for j in range(label.shape[1]):
            if(label[i, j] == 1):
                ll[i, j] = img[i, j]
    return ll


In [8]:
def clip_image(img):
    top = max(np.where(img != 0)[0])
    down = min(np.where(img != 0)[0])
    r = max(np.where(img != 0)[1])
    l = min(np.where(img != 0)[1])
    return img[down:top+1, l:r+1]

In [9]:
#each input slide is 100*100 vector, and a input include 3 slides(for CBD, SMV, Retro respectively)

import cv2
import math

#定义最大灰度级数
gray_level = 3001

def maxGrayLevel(img):
    max_gray_level=0
    (height,width)=img.shape
    for y in range(height):
        for x in range(width):
            if img[y][x] > max_gray_level:
                max_gray_level = img[y][x]
    return max_gray_level+1

def getGlcm(input,d_x,d_y):
    srcdata=input.copy()
    max_value = np.max(srcdata)
    min_value = np.min(srcdata)
    for i in range(srcdata.shape[0]):
        for j in range(srcdata.shape[1]):
            srcdata[i, j] = int((srcdata[i, j] - min_value)/(max_value - min_value)*(gray_level - 1))
#     print(srcdata)
    ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
    (height,width) = input.shape
    
    max_gray_level=maxGrayLevel(input)
    
    #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
    
#     if max_gray_level > gray_level:
#         for j in range(height):
#             for i in range(width):
#                 srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level

    for j in range(height-abs(d_y)):
        for i in range(width-abs(d_x)):
            rows = int(srcdata[j][i])
            cols = int(srcdata[j + d_y][i+d_x])
            ret[rows][cols]+=1.0

#     for i in range(gray_level):
#         for j in range(gray_level):
#             ret[i][j]/=float(height*width)

    return np.array(ret)

In [10]:
new_y = []

# Establish The npz data (3 glcm slide for each input)
from keras.utils import to_categorical
from scipy import ndimage
type_name_pd = pd.DataFrame(info.groupby(['resection', 'SMV', 'CBD',  'retroperitoneal']).case_id)

#create [name, index, y], notice that y = vector(5)
kk = 0
num_type = len(type_name_pd) - 1
for i in range(num_type):    #neglect x, x, x
    vec5 = [0 for i in range(num_type)]
    vec5[i] = 1
    n = len(type_name_pd[1][i])
    total_val = 0
    for j in range(n):
        name = type_name_pd[1][i].iloc[j]
        
        name_idx = np.where(resec == name)[0][0]
        image = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))
        label = load_image(os.path.join(label_path, "LB_"+name+".nii.gz"))
        print(name)
        for k in range(int(box_tumor[name_idx, 4]), int(box_tumor[name_idx, 5])+1):
            
            tmp_slide = label[..., k]
            tmp_slide_img = image[..., k]
            if(np.where(tmp_slide == 2)[0].size == 0):
                continue
            
            
            new_y.append(vec5)
            kk+=1
            
            
        
        del image, label
print("created new_y")

PT7
PC607
PC602
PC600
PC593
PC573
PC570
PC561
PC536
PC595
PC580
PC556
PC540
PC768
PC42
PC104
PC285
PC636
PC535
PC563
PC519
PC447
PC484
PC898
PC818
PC699
PC769
PC739
PT12
PC581
PC544
PC537
PT8
PC116
PC545
PC583
PC456
PC477
created new_y


In [12]:
type_name_pd = pd.DataFrame(info.groupby(['resection', 'SMV', 'CBD',  'retroperitoneal']).case_id)

#create [name, index, y], notice that y = vector(5)
io_arr = [[] for i in range(3)]
num_type = len(type_name_pd) - 1
for i in range(num_type):    #neglect x, x, x
    vec5 = [0 for i in range(num_type)]
    vec5[i] = 1
    n = len(type_name_pd[1][i])
    total_val = 0
    for j in range(n):
        name = type_name_pd[1][i].iloc[j]
        name_idx = np.where(resec == name)[0][0]

        image = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))
        label = load_image(os.path.join(label_path, "LB_"+name+".nii.gz"))

        for k in range(int(box_tumor[name_idx, 4]), int(box_tumor[name_idx, 5])+1):
            
            tmp_slide = label[..., k]
            tmp_slide_img = image[..., k]
            if(np.where(tmp_slide == 2)[0].size == 0):
                continue
            io_arr[0].append(name)
            io_arr[1].append(k)
            io_arr[2].append(vec5)
        del image, label
print("create io_arr")

create io_arr


In [13]:
order_arr = np.zeros((len(io_arr[0]), 1))
for i in range(order_arr.shape[0]):
    order_arr[i] = i
import random


#random.shuffle(order_arr)
np.random.shuffle(order_arr)
np.unique(order_arr)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [14]:
from keras.models import Model#, load_model
from keras import layers as klayers
from keras.optimizers import Adam
from keras import utils as kutils
from keras import backend as K
from keras.callbacks import ModelCheckpoint

from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
#from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
#import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

#init_notebook_mode(connected=True)
%matplotlib inline


from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate,  MaxPool2D, Reshape
# from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector, Reshape
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [15]:
import pandas as pd
map_df = pd.DataFrame(data={'resec_name': io_arr[0], 'idx': io_arr[1], 'target':io_arr[2], 'order' : order_arr.reshape(len(io_arr[0]))})

In [16]:
from keras.utils import to_categorical
from scipy import ndimage
class resec_Generator1(kutils.Sequence):
    'Generates data for Keras'
    def __init__(self, mapping_df, batch_size, shuffle=True):
        'Initialization'
        self.mapping_df = mapping_df
        self.data_num   = mapping_df.shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data_num / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        #print("enter0")
        batch_mapping_df = \
            self.mapping_df.iloc[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(batch_mapping_df)
        return X, y


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            self.mapping_df = self.mapping_df.sample(frac=1).reset_index(drop=True)
            
    def __data_generation(self, batch_mapping_df):
        'Generates data containing batch_size samples' 
        # Initialization
        X = np.zeros((  self.batch_size, 3000, 3000, 3, 1)) 


        y = np.zeros((self.batch_size, 5))

        # Generate data
        cnt = 0
        
        for i, row in batch_mapping_df.iterrows():
            #img = extract_cube(shape_tumor[row['shape']], load_image(os.path.join(image_path, "IM_"+row['resec_name']+".nii.gz")))
            idx = int(row['order'])#encode = row['resec_name']
            encode = io_arr[0][idx]
            image = load_image(os.path.join(image_path, "IM_"+encode+".nii.gz"))
            label = load_image(os.path.join(label_path, "LB_"+encode+".nii.gz"))
            k = io_arr[1][idx]
            tmp_slide = label[..., k]
            tmp_slide_img = image[..., k]
            if(np.where(tmp_slide == 2)[0].size == 0):
                continue
            down = max(np.where(tmp_slide == 2)[0])
            top = min(np.where(tmp_slide == 2)[0])
            r = max(np.where(tmp_slide == 2)[1])
            l = min(np.where(tmp_slide == 2)[1])
            center = int((down + top)/2), int((r + l)/2)
            dilation_pancreas = ndimage.binary_dilation(tmp_slide, iterations = 10).astype(tmp_slide.dtype)
            erosion_tumor = ndimage.binary_erosion(extract_tumor(tmp_slide), iterations = 1).astype(tmp_slide.dtype)
            final_masking_label = masking_dilation_erosion(dilation_pancreas, erosion_tumor)
            CBD = final_masking_label[:center[0], :center[1]]
            CBD_tmp_slide_img = tmp_slide_img[:center[0], :center[1]]

            SMV = final_masking_label[:center[0], center[1]:]
            SMV_tmp_slide_img = tmp_slide_img[:center[0], center[1]:]

            Retro = final_masking_label[center[0]:, :]
            Retro_tmp_slide_img = tmp_slide_img[center[0]:, :]

            
            CBD_image = masking_image(CBD, CBD_tmp_slide_img)
            SMV_image = masking_image(SMV, SMV_tmp_slide_img)
            Retro_image = masking_image(Retro, Retro_tmp_slide_img)
            
            CBD_image = clip_image(CBD_image)
            SMV_image = clip_image(SMV_image)
            Retro_image = clip_image(Retro_image)

            CBD_image = getGlcm(CBD_image,-1,1)
            SMV_image = getGlcm(SMV_image,1,1)
            Retro_image = getGlcm(Retro_image,0,-1)
            
            glcm_input = np.concatenate([np.expand_dims(getGlcm(CBD_image,-1,1), axis = 2), np.expand_dims(getGlcm(SMV_image,1,1), axis = 2), np.expand_dims(getGlcm(Retro_image,0,-1), axis = 2)], axis = 2)

            #print(encode)
            X[ cnt, :, :, :, 0] = glcm_input[:3000, :3000, :]
            del image, label
            y[cnt, :] = io_arr[2][idx]


            cnt += 1
        #print(X[5])
        return X, y

# for name in resec:
#     img = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))

In [17]:
from keras import metrics
tmp_x = Input(( 3000, 3000, 3, 1))

x = Reshape(( 300, 300, 300, 1))(tmp_x)
x = Conv3D(filters=16, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu',kernel_initializer = keras.initializers.he_normal(seed=None))(x)
x = MaxPool3D()(x)
x = MaxPool3D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = MaxPool3D()(x)



x = Conv3D(filters=8, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu',kernel_initializer = keras.initializers.he_normal(seed=None))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = MaxPool3D()(x)

x = Conv3D(filters=8, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu',kernel_initializer = keras.initializers.he_normal(seed=None))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = MaxPool3D()(x)

# x = Conv3D(filters=4, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu',kernel_initializer = keras.initializers.he_normal(seed=None))(tmp_x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Conv3D(filters=2, dilation_rate=1, kernel_size=(3, 3, 3), activation='relu',kernel_initializer = keras.initializers.he_normal(seed=None))(tmp_x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)



x = Flatten()(x)
x = Dense(units=100, kernel_initializer = keras.initializers.he_normal(seed=None), activation='elu')(x)
x = Dense(units=50, activation='selu')(x)
x = Dense(units=10, activation='relu')(x)




x = Dense(5, activation='sigmoid')(x)





model = Model(inputs=tmp_x, outputs=x)
adam_lr = 0.0003
adam_beta_1 = 0.5
model.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
              loss='categorical_crossentropy', metrics = [metrics.categorical_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3000, 3000, 3, 1)  0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 300, 300, 300, 1)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 298, 298, 298, 16) 448       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 149, 149, 149, 16) 0         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 74, 74, 74, 16)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 74, 74, 74, 16)    64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 74, 16)    0         
__________

In [18]:
batch_size = 1
v_generator = resec_Generator1(map_df[:-20], batch_size=batch_size)
validation_generator = resec_Generator1(map_df[-20:], batch_size=batch_size)

In [19]:
history = model.fit_generator(v_generator,
                              epochs=100, validation_data=validation_generator)

Epoch 1/100
159/159 [==============================] - 12390s 78s/step - loss: 1.8269 - categorical_accuracy: 0.2704 - val_loss: 1.5584 - val_categorical_accuracy: 0.1500
Epoch 2/100
159/159 [==============================] - 11420s 72s/step - loss: 1.5772 - categorical_accuracy: 0.2642 - val_loss: 1.5702 - val_categorical_accuracy: 0.2000
Epoch 3/100
159/159 [==============================] - 11611s 73s/step - loss: 1.5039 - categorical_accuracy: 0.3522 - val_loss: 1.5945 - val_categorical_accuracy: 0.1000
Epoch 4/100
159/159 [==============================] - 11469s 72s/step - loss: 1.4695 - categorical_accuracy: 0.4025 - val_loss: 1.5827 - val_categorical_accuracy: 0.0500
Epoch 5/100
159/159 [==============================] - 11590s 73s/step - loss: 1.5431 - categorical_accuracy: 0.3899 - val_loss: 1.5884 - val_categorical_accuracy: 0.1000
Epoch 6/100
159/159 [==============================] - 11284s 71s/step - loss: 1.4492 - categorical_accuracy: 0.3836 - val_loss: 1.5820 - val_cat

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



159/159 [==============================] - 11676s 73s/step - loss: 1.2855 - categorical_accuracy: 0.4340 - val_loss: 1.4886 - val_categorical_accuracy: 0.2500
Epoch 53/100
159/159 [==============================] - 11474s 72s/step - loss: 1.3146 - categorical_accuracy: 0.4403 - val_loss: 1.4740 - val_categorical_accuracy: 0.4000
Epoch 54/100
159/159 [==============================] - 11512s 72s/step - loss: 1.3069 - categorical_accuracy: 0.4151 - val_loss: 1.5249 - val_categorical_accuracy: 0.2500
Epoch 55/100
159/159 [==============================] - 11512s 72s/step - loss: 1.2791 - categorical_accuracy: 0.4717 - val_loss: 1.5314 - val_categorical_accuracy: 0.2500
Epoch 56/100
159/159 [==============================] - 11353s 71s/step - loss: 1.3130 - categorical_accuracy: 0.4528 - val_loss: 1.5398 - val_categorical_accuracy: 0.2500
Epoch 57/100
159/159 [==============================] - 11511s 72s/step - loss: 1.3454 - categorical_accuracy: 0.4214 - val_loss: 1.5176 - val_categorica

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [21]:
history = model.fit_generator(v_generator,
                              epochs=30, validation_data=validation_generator)

Epoch 1/30
159/159 [==============================] - 12123s 76s/step - loss: 1.2567 - categorical_accuracy: 0.4591 - val_loss: 1.5015 - val_categorical_accuracy: 0.3000
Epoch 2/30
159/159 [==============================] - 11503s 72s/step - loss: 1.2203 - categorical_accuracy: 0.4843 - val_loss: 1.4879 - val_categorical_accuracy: 0.3000
Epoch 3/30
159/159 [==============================] - 11388s 72s/step - loss: 1.2377 - categorical_accuracy: 0.4591 - val_loss: 1.4927 - val_categorical_accuracy: 0.3000
Epoch 4/30
159/159 [==============================] - 11372s 72s/step - loss: 1.1909 - categorical_accuracy: 0.5157 - val_loss: 1.4913 - val_categorical_accuracy: 0.3500
Epoch 5/30
159/159 [==============================] - 11520s 72s/step - loss: 1.2556 - categorical_accuracy: 0.4528 - val_loss: 1.5176 - val_categorical_accuracy: 0.2500
Epoch 6/30
159/159 [==============================] - 11564s 73s/step - loss: 1.3339 - categorical_accuracy: 0.4591 - val_loss: 1.5212 - val_categoric

KeyboardInterrupt: 

In [22]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
history = model.fit_generator(v_generator,
                              epochs=30, validation_data=validation_generator)

Epoch 1/30
159/159 [==============================] - 12136s 76s/step - loss: 1.2338 - categorical_accuracy: 0.4340 - val_loss: 1.4873 - val_categorical_accuracy: 0.3000
Epoch 2/30
159/159 [==============================] - 11458s 72s/step - loss: 1.2354 - categorical_accuracy: 0.4654 - val_loss: 1.5100 - val_categorical_accuracy: 0.2500
Epoch 3/30
159/159 [==============================] - 11456s 72s/step - loss: 1.2151 - categorical_accuracy: 0.4465 - val_loss: 1.4727 - val_categorical_accuracy: 0.3500
Epoch 4/30
159/159 [==============================] - 11347s 71s/step - loss: 1.2195 - categorical_accuracy: 0.4717 - val_loss: 1.4730 - val_categorical_accuracy: 0.4000
Epoch 5/30
159/159 [==============================] - 11508s 72s/step - loss: 1.2185 - categorical_accuracy: 0.4340 - val_loss: 1.4684 - val_categorical_accuracy: 0.4000
Epoch 6/30
159/159 [==============================] - 11619s 73s/step - loss: 1.2399 - categorical_accuracy: 0.4717 - val_loss: 1.5050 - val_categoric